# Wörterbuch-Codierung
## 1. Aufgabenstellung
In dieser Laboraufgabe wollen wir die beiden Verfahren Lempel-Ziv (LZ77) und Lempel-Ziv-
Welch (LZW) implementieren und vergleichen. Finden Sie sich daher wieder wie bei der
Eintropie-Codierung zu Team-Paaren zusammen, von denen ein Team die Codierung und
Decodierung nach Lempel-Ziv und nach Lempel-Ziv-Welch implementiert.
## 2. Lempel-Ziv
Implementieren Sie eine Klasse `LempelZiv`, die die Codierung nach dem Lempel-Ziv-Verfahren
durchführt. Die Klasse erhält als Attribute
- die Anzahl Bits für die Codierung der Rückwärtsreferenz
- die Anzahl Bits für die Länge der Zeichenkette

Die Klasse soll über die folgenden Methoden verfügen:
- `bitstring, charD = encode(message)`
- `message = decode(bitstring,charD)`
Das Dictionary `charD` enthält die verwendete Codierung der Buchstaben zu Bits. Codieren Sie alle
Buchstaben mit der gleichen Anzahl von Bits, die Sie anhand der in der Nachricht vorkommenden
Buchstaben festlegen können.

In [ ]:
class LempelZiv:
    def __init__(self, offset_bits, length_bits):
        self.offset_bits = offset_bits
        self.length_bits = length_bits

    def encode(self, message):
        # Create character dictionary with fixed bit length for all characters
        unique_chars = sorted(set(message))
        bits_needed = len(bin(len(unique_chars) - 1)[2:])  # Minimum bits required
        char_dict = {
            char: format(i, f"0{bits_needed}b") for i, char in enumerate(unique_chars)
        }

        result = []
        pos = 0

        while pos < len(message):
            # Find the longest match in the already processed text
            best_offset = 0
            best_length = 0

            # Search for matches within the window size limit
            search_start = max(0, pos - (2**self.offset_bits - 1))
            for i in range(search_start, pos):
                # Try to match from this position
                match_length = 0
                while (
                    pos + match_length < len(message)
                    and i + match_length < pos
                    and message[i + match_length] == message[pos + match_length]
                    and match_length < (2**self.length_bits - 1)
                ):
                    match_length += 1

                if match_length > best_length:
                    best_length = match_length
                    best_offset = pos - i

            if best_length > 0:
                # Add backward reference flag (1)
                result.append("1")
                # Add offset in binary
                result.append(format(best_offset, f"0{self.offset_bits}b"))
                # Add length in binary
                result.append(format(best_length, f"0{self.length_bits}b"))
                pos += best_length
            else:
                # Add literal character flag (0)
                result.append("0")
                # Add character code
                result.append(char_dict[message[pos]])
                pos += 1

        return "".join(result), char_dict

    def decode(self, bitstring, char_dict):
        # Create reverse mapping for decoding
        rev_dict = {code: char for char, code in char_dict.items()}
        char_bits = len(next(iter(char_dict.values())))

        result = []
        pos = 0

        while pos < len(bitstring):
            flag = bitstring[pos]
            pos += 1

            if flag == "0":
                # Process literal character
                char_code = bitstring[pos : pos + char_bits]
                result.append(rev_dict[char_code])
                pos += char_bits
            else:
                # Process backward reference
                offset = int(bitstring[pos : pos + self.offset_bits], 2)
                pos += self.offset_bits
                length = int(bitstring[pos : pos + self.length_bits], 2)
                pos += self.length_bits

                # Copy the referenced text
                start_pos = len(result) - offset
                for i in range(length):
                    result.append(result[start_pos + i])

        return "".join(result)

## 3. Lempel-Ziv-Welch
Implementieren Sie eine Klasse LempelZivWelch, die die Codierung nach dem Lempel-Ziv-
Welche-Verfahren durchführt.
Die Klasse soll über die folgenden Methoden verfügen:
- `bitstring=encode(message,charL)`
- `message=decode(bitstring, charL)`

Die Liste `charL` enthält die Zeichen, mit denen das Wörterbuch initialisiert werden soll.
Verwenden Sie in der Aufgabe nur die in der Nachricht vorkommenden Zeichen.

In [ ]:
class LempelZivWelch:
    def __init__(self):
        pass

    def encode(self, message, charL):
        # Initialize dictionary with characters from charL
        dictionary = {char: i for i, char in enumerate(charL)}
        dict_size = len(dictionary)

        # Calculate minimum bits needed to represent dictionary indices
        # Ensure at least 1 bit is used
        bits_per_code = max(1, len(bin(dict_size - 1)[2:]))

        # Initialize variables
        result = []
        current = ""

        # Process each character in the message
        for char in message:
            combined = current + char
            if combined in dictionary:
                current = combined
            else:
                # Output code for current string
                code = dictionary[current]
                result.append(format(code, f"0{bits_per_code}b"))

                # Add new sequence to dictionary
                dictionary[combined] = dict_size
                dict_size += 1

                # Update bits per code if dictionary size increases beyond current bit width
                if dict_size > (1 << bits_per_code):
                    bits_per_code += 1

                current = char

        # Don't forget the last code
        if current:
            code = dictionary[current]
            result.append(format(code, f"0{bits_per_code}b"))

        # Combine all binary codes into a single bitstring
        return "".join(result)

    def decode(self, bitstring, charL):
        if not bitstring:
            return ""

        # Initialize dictionary with characters from charL
        dictionary = {i: char for i, char in enumerate(charL)}
        dict_size = len(dictionary)

        # Calculate initial bits per code (ensure at least 1 bit)
        bits_per_code = max(1, len(bin(dict_size - 1)[2:]))

        # Process the first code
        pos = 0
        current_code = int(bitstring[:bits_per_code], 2)
        pos += bits_per_code

        result = dictionary[current_code]
        previous_string = result

        # Process the rest of the bitstring
        while pos < len(bitstring):
            # Check if we need to increase bits per code
            if dict_size == (1 << bits_per_code):
                bits_per_code += 1

            # Extract next code
            if pos + bits_per_code <= len(bitstring):
                code = int(bitstring[pos : pos + bits_per_code], 2)
                pos += bits_per_code
            else:
                # Handle potential padding at the end
                break

            # Handle the dictionary lookup
            if code in dictionary:
                entry = dictionary[code]
            elif code == dict_size:
                # Special case: the next code is one we're about to create
                # This happens when the pattern is like ww where w is the previous string
                entry = previous_string + previous_string[0]
            else:
                raise ValueError(f"Invalid LZW code: {code}")

            # Append the entry to the result
            result += entry

            # Add new sequence to dictionary
            dictionary[dict_size] = previous_string + entry[0]
            dict_size += 1

            # Update previous string for next iteration
            previous_string = entry

        return result

## 4. Test
Überprüfen Sie ihre Implementierung anhand der Beispiele aus der Vorlesung.

In [42]:
"""Tests"""
test_messages = ["FISCHERSFRITZFISCHTFRISCHEFISCHE", "BANANENANBAU", "XXXXXX"]

for message in test_messages:
    print(f"Original message: {message}")

    # Lempel-Ziv encoding and decoding
    lz = LempelZiv(12, 8)
    encoded_lz, charD = lz.encode(message)
    print(f"Lempel-Ziv Encoded: {encoded_lz}")
    decoded_lz = lz.decode(encoded_lz, charD)
    print(f"Lempel-Ziv Decoded: {decoded_lz}")

    # Lempel-Ziv-Welch encoding and decoding
    lzw = LempelZivWelch()
    charL = sorted(set(message))
    encoded_lzw = lzw.encode(message, charL)
    print(f"Lempel-Ziv-Welch Encoded: {encoded_lzw}")
    decoded_lzw = lzw.decode(encoded_lzw, charL)
    print(f"Lempel-Ziv-Welch Decoded: {decoded_lzw}")

    print("-" * 40)

Original message: FISCHERSFRITZFISCHTFRISCHEFISCHE
Lempel-Ziv Encoded: 000100010000110000000001100001001011000000000101000000011000000001000000000011000000000011000000011000000001001000000010011101000100000000110100000101100000000011100000001100000000101100000011100000001010000000100100000001101000000110
Lempel-Ziv Decoded: FISCHERSFRITZFISCHTFRISCHEFISCHE
Lempel-Ziv-Welch Encoded: 00100100011000000011000101010110000100010100100001110100001001010110001100111100010101001100000011011011100
Lempel-Ziv-Welch Decoded: FISCHERSFRITZFISCHTFRISCHEFISCHE
----------------------------------------
Original message: BANANENANBAU
Lempel-Ziv Encoded: 00010000001110000000000100000001000101000000000100000000111000000001001000000100100
Lempel-Ziv Decoded: BANANENANBAU
Lempel-Ziv-Welch Encoded: 00100001111000100111001101010100
Lempel-Ziv-Welch Decoded: BANANENANBAU
----------------------------------------
Original message: XXXXXX
Lempel-Ziv Encoded: 0010000000000010000000110000000000100000001010000000001

## 5. Performance
### 5.1 Einfluss der Codierung bei Lempel-Ziv
Bestimmen Sie für unsere Test-Nachricht, den RFC 2324, die Kombination aus Anzahl Bits für
Rückwärtsreferenz und Zeichenkettenlänge, die das optimale Ergebnis liefert.

In [ ]:
htcpcp = (
    "BREW\r\n"
    "Scheme: coffee\r\n"
    "Content-Type: message/coffeepot\r\n"
    "Accept-Additions: milk-type/Cream, syrup-type/Vanilla\r\n"
    "coffee-message-body=start\r\n"
)

optimal_bits = None
min_length = float("inf")

for bits_reference in range(1, 50):
    for bits_length in range(1, 50):
        lz = LempelZiv(bits_reference, bits_length)
        encoded_lz, charD = lz.encode(htcpcp)
        length = len(encoded_lz)

        if length < min_length and lz.decode(encoded_lz, charD) == htcpcp:
            min_length = length
            optimal_bits = (bits_reference, bits_length)

print(
    f"Optimale Parameter:\nRückwärtsreferenz-Bits: {optimal_bits[0]}\nZeichenkettenlänge-Bits: {optimal_bits[1]}"
)
print(f"Minimale Länge: {min_length}")

print(
    f"Von: {len(htcpcp) * 8} auf {min_length} Bits. Ratio: {min_length / (len(htcpcp) * 8):.2f}"
)

Optimale Parameter:
Rückwärtsreferenz-Bits: 3
Zeichenkettenlänge-Bits: 1
Minimale Länge: 893
Von: 1096 auf 893 Bits. Ratio: 0.81


### 5.2 Vergleich
Wenden Sie die beiden Verfahren auf unsere Test-Nachricht an, den RFC 2324. Welche ist besser?

In [46]:
"""Vergleich der Algorithmen"""
lz = LempelZiv(3, 1)
encoded_lz, charD = lz.encode(htcpcp)

lzw = LempelZivWelch()
charL = sorted(set(htcpcp))
encoded_lzw = lzw.encode(htcpcp, charL)

assert lz.decode(encoded_lz, charD) == htcpcp, "Lempel-Ziv Decoding failed"
assert lzw.decode(encoded_lzw, charL) == htcpcp, "Lempel-Ziv-Welch Decoding failed"

print(f"Originallänge: {len(htcpcp) * 8} Bits")
print(f"Lempel-Ziv Länge: {len(encoded_lz)} Bits")
print(f"Lempel-Ziv-Welch Länge: {len(encoded_lzw)} Bits")
print(f"Ratio Lempel-Ziv: {len(encoded_lz) / (len(htcpcp) * 8):.2f}")
print(f"Ratio Lempel-Ziv-Welch: {len(encoded_lzw) / (len(htcpcp) * 8):.2f}")

Originallänge: 1096 Bits
Lempel-Ziv Länge: 893 Bits
Lempel-Ziv-Welch Länge: 776 Bits
Ratio Lempel-Ziv: 0.81
Ratio Lempel-Ziv-Welch: 0.71
